 # Import necessary packages including GTITAN

In [1]:
from GTITAN import GTITAN
import pandas as pd
import imp
import numpy as np
from functools import reduce
import os

a=GTITAN()
location_data='C:\\Users\\adema\\Documents\\ifo\\ifo\\Coursework\\ML\\Term Paper Project\\'

Using directory2 'C:\Users\adema\Documents\ifo\ifo\Coursework\ML\Term Paper Project\GTITAN\GTITAN' C:\Users\adema\Documents\ifo\ifo\Coursework\ML\Term Paper Project\GTITAN\GTITAN\core.py
Using directory 'C:\Users\adema\Anaconda3\lib\site-packages\gtab'
Active anchorbank changed to: google_anchorbank_geo=_timeframe=2019-01-01 2020-08-01.tsv



 # Importing and modifying the IBC monthly data

In [2]:
#Import IBC monthly data
flow_data = pd.read_excel(location_data+'Data\\Monthly_detections_of_IBC_2021_01_05.xlsx',sheet_name=0)
flow_data['Nationality'] = flow_data['Nationality'].astype(str)
flow_data['Nationality'] = flow_data['Nationality'].str.replace("Cape Verde",'Cabo Verde').str.replace('Central African Republic','Central African Republic (the)').str.replace("Nigeria","Nigtemp").str.replace("Niger","Niger (the)").str.replace("Nigtemp","Nigeria").str.replace("South Sudan","Ssudtemp").str.replace("Sudan","Sudan (the)").str.replace("Ssudtemp","South Sudan").str.replace("Congo (Brazzaville)","Congo (the)").str.replace("Congo (Kinshasa)","Congo (the Democratic Republic of the)").str.replace("Gambia","Gambia (the)").str.replace("Syria","Syrian Arab Republic").str.replace("Comoros","Comoros (the)").str.replace("Tanzania","Tanzania, United Republic of").str.replace("Philippines","Philippines (the)").str.replace("Russia","Russian Federation (the)").str.replace("Iran","Iran (Islamic Republic of)").str.replace("Moldova","Moldova (the Republic of)").str.replace("Dominican Republic","Dominican Republic (the)").str.replace("Namibia","Namibia").str.replace("Palestine^","Palestine, State of").str.replace("Bolivia","Bolivia (Plurinational State of)").str.replace("Vietnam","Viet Nam").str.replace("Lao People's Democratic Republic","Lao People's Democratic Republic (the)").str.replace("Swaziland","Eswatini").str.replace("North Macedonia","Republic of North Macedonia").str.replace("Taiwan","Taiwan (Province of China)").str.replace("Venezuela","Venezuela (Bolivarian Republic of)").str.replace("United Arab Emirates","United Arab Emirates (the)")
iso_data = pd.read_excel(location_data+'Data\\ISO23.xlsx',sheet_name=0)

#Merge ISO identifiers to be able to query consituency and repair some errors:
flow2 = flow_data.merge(iso_data,right_on='Name',left_on='Nationality',how='left')
flow2 = flow2[(flow2['Nationality']!='United States')&(flow2['Nationality']!='South Korea')&(flow2['Nationality']!='North Korea')&(flow2['Nationality']!='Saint Helena')&(flow2['Nationality']!='Unknown') &(flow2['Nationality']!='Pitcairn')  &(flow2['Nationality']!='Unspecified sub-Saharan nationals') &(flow2['Nationality']!='Stateless') &(flow2['Nationality']!='Third-country') &(flow2['Nationality']!='Bahamas')] 
flow2[flow2['ISO2'].isna()]['Nationality'].unique()
flow2['ISO2'][flow2['Nationality']=='Kosovo']='XK'
flow2['ISO2'][flow2['Nationality']=='Namibia']='NA'
flow2['ISO3'][flow2['Nationality']=='Namibia']='NAM'

#remove countries we wont use
remove_list = ['Gibraltar','Cambodia','India','Bermuda','Lesotho','Dominica','Panama','Malaysia','French Guiana','Israel','Haiti','Jamaica','Belize','Australia','Canada','Colombia','Cuba','Dominican Republic (the)','Japan','Ecuador','Isle of Man','Macao','New Zealand','Papua New Guinea','Réunion','Chile','Samoa','Singapore','Solomon Islands','Brazil','Argentina','Maldives','Kiribati','Guyana','Seychelles','United Arab Emirates (the)','Venezuela (Bolivarian Republic of)','Suriname','Hong Kong','Paraguay']
flow2 = flow2[~flow2['Nationality'].isin(remove_list)]

#remove countries that are problematic in terms of anchorbanking, have a look at this later:
remove_list = ['KH','MN','LR','MG']
flow2 = flow2[~flow2['ISO2'].isin(remove_list)]

#aggregate over land/sea, as it just says something about detecton pt
flow2 = flow2.groupby(['Route','Nationality','Language','ISO2']).sum().reset_index()

<ipython-input-2-dd28e20ef667>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flow2['ISO2'][flow2['Nationality']=='Namibia']='NA'


 ## Source GTITAN

In [6]:
#the list of countries in scope and their local languages part of Google Translate: 
#number of languages should be lower than 5 to meet the GTI keyword limit.
trans_dict = {"AF":[["Persian",'fa'],["Pashto",'ps']],"DZ":[["Arabic",'ar']],"EG":[["Arabic",'ar']],"JO":[["Arabic",'ar']],"KW":[["Arabic",'ar']],"LB":[["Arabic",'ar']],"LY":[["Arabic",'ar']],"MR":[["Arabic",'ar']],"OM":[["Arabic",'ar']],"PS":[["Arabic",'ar']],"SA":[["Arabic",'ar']],"SD":[["Arabic",'ar']],"TN":[["Arabic",'ar']],"YE":[["Arabic",'ar']],"BY":[["Belarusian",'be']],"IR":[["Persian",'fa']],"IQ":[["Arabic",'ar'],["Kurdish",'ku']],"SY":[["Arabic",'ar'],["Kurdish",'ku']],"MA":[["French",'fr'],["Arabic",'ar']],"TD":[["Arabic",'ar'],["French",'fr']],"KM":[["French",'fr'],["Arabic",'ar']],"DJ":[["French",'fr'],["Arabic",'ar']],"PK":[["Urdu",'ur'],["English",'en']],"RU":[["Russian",'ru']],"SO":[["Somali",'so'],["Arabic",'ar']],"TR":[["Turkish",'tr']],"UA":[["Ukrainian",'uk'],["Russian",'ru']],"AL":[["Albanian",'sq']],"AO":[["Portuguese",'pt']],"CV":[["Portuguese",'pt']],"MZ":[["Portuguese",'pt']],"AZ":[["Azerbaijani",'az']],"BD":[["Bengali",'bn']],"BJ":[["French",'fr']],"BF":[["French",'fr']],"BI":[["French",'fr']],"CF":[["French",'fr']],"CD":[["French",'fr']],"CG":[["French",'fr']],"CI":[["French",'fr']],"GA":[["French",'fr']],"GN":[["French",'fr']],"ML":[["French",'fr']],"NE":[["French",'fr']],"TG":[["French",'fr']],"SN":[["French",'fr']],"CM":[["French",'fr'],["English",'en']],"CN":[["Mandarin",'zh-CN']],"TW":[["Mandarin",'zh-CN']],"GQ":[["French",'fr'],["Spanish",'es'],["Portuguese",'pt']],"ET":[["English",'en'],["Amharic",'am']],"GM":[["English",'en']],"GH":[["English",'en']],"MW":[["English",'en']],"NA":[["English",'en']],"SL":[["English",'en']],"SS":[["English",'en']],"ZM":[["English",'en']],"ZW":[["English",'en']],"GE":[["Georgian",'ka']],"GW":[["Portuguese",'pt'],["French",'fr']],"KZ":[["Kazakh",'kk'],["Russian",'ru']],"KE":[["Swahili",'sw'],["English",'en']],"UG":[["Swahili",'sw'],["English",'en']],"XK":[["Albanian",'sq'],["Croatian",'hr']],"KG":[["Kyrgyz",'ky'],["Russian",'ru']],"MU":[["English",'en'],["French",'fr']],"MD":[["Romanian",'ro'],["Russian",'ru']],"MM":[["Burmese",'my']],"NP":[["Nepali",'ne']],"NG":[["English",'en'],["Hausa",'ha'],["Igbo",'ig']],"PH":[["English",'en'],["Filipino",'tl'],["Cebuano",'ceb']],"RW":[["French",'fr'],["English",'en'],["Swahili",'sw']],"RS":[["Croatian",'hr']],"BA":[["Croatian",'hr']],"ME":[["Croatian",'hr']],"ZA":[["English",'en'],["Afrikaans",'af']],"TJ":[["Tajiki",'tg']],"TZ":[["English",'en'],["Swahili",'sw']],"UZ":[["Uzbek",'uz'],["Russian",'ru']],"VN":[["Vietnamese",'vi']],"AM":[["Armenian",'hy']],"MX":[["Spanish",'es']],"PE":[["Spanish",'es']],"MK":[["Macedonian",'mk']],"LK":[["Sinhala",'si'],["Tamil",'ta'],["English",'en']],"TM":[["Turkmen",'tk']],"ID":[["Indonesian",'id']],"LA":[["Lao",'lo']]}

#Countries to query, keywords to query and call main_flex of GTITAN
origin_list = flow2['ISO2'].unique()
keyword_list_immigeneral = ["legalization","passport","visa","quota","waiver","immigrant","immigrate","immigration","arrival","emigrant","emigrate","emigration","required documents","migrant","migrate","migration"]
roi_df = pd.DataFrame({'regions':origin_list})
gprop_df =  pd.DataFrame({'gprop':['']})
key_df = pd.DataFrame({'keys':keyword_list_immigeneral})
cat_df = pd.DataFrame({'cat':['0']})
timeframe_df = pd.DataFrame({'timeframe':['all']})
time_df = a.main_flex(trans_dict,roi_df,gprop_df,key_df,cat_df,timeframe_df,raise_IP=True,newdf=False)
time_df.to_csv(location_data+'Data\\timeframe.csv')


Contains English already
Contains English already
Contains English already
Contains English already
Contains English already
Contains English already
Contains English already
Contains English already
Contains English already
Contains English already
Contains English already
Contains English already
Contains English already
Contains English already
Contains English already
Contains English already
Contains English already
Contains English already
Contains English already
Contains English already
Frame loaded!
We have already translated this term
We have already translated this term
We have already translated this term
We have already translated this term
We have already translated this term
We have already translated this term
We have already translated this term
We have already translated this term
We have already translated this term
We have already translated this term
We have already translated this term
We have already translated this term
We have already translated this term
We ha

We have already translated this term
We have already translated this term
We have already translated this term
We have already translated this term
We have already translated this term
We have already translated this term
We have already translated this term
We have already translated this term
We have already translated this term
We have already translated this term
We have already translated this term
We have already translated this term
We have already translated this term
We have already translated this term
We have already translated this term
We have already translated this term
We have already translated this term
We have already translated this term
We have already translated this term
We have already translated this term
We have already translated this term
We have already translated this term
We have already translated this term
We have already translated this term
We have already translated this term
We have already translated this term
We have already translated this term
W

The DF is empty: no match found. We create an empty DF as a placeholder with same dimensions.
cols_to_drop:  ['قانونی شدن ' 'قانوني کول ' 'legalization '] columns of trendlist: ['legalization ', 'قانوني کول ', 'قانونی شدن ', 'isPartial', 'cat', 'geo', 'gprop', 'legalization']
The DF is empty: no match found. We create an empty DF as a placeholder with same dimensions.
cols_to_drop:  ['مدارک مورد نیاز ' 'اړین لاسوندونه ' 'required documents '] columns of trendlist: ['اړین لاسوندونه ', 'required documents ', 'مدارک مورد نیاز ', 'isPartial', 'cat', 'geo', 'gprop', 'required documents']
The DF is empty: no match found. We create an empty DF as a placeholder with same dimensions.
cols_to_drop:  ['required documents ' 'dokumentet e kërkuara '] columns of trendlist: ['required documents ', 'dokumentet e kërkuara ', 'isPartial', 'cat', 'geo', 'gprop', 'required documents']
The DF is empty: no match found. We create an empty DF as a placeholder with same dimensions.
cols_to_drop:  ['required do

ReadTimeout: HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)

 ## Merge into a DataFrame suitable for an LSTM

In [8]:
time_df=pd.read_csv(location_data+'Data\\timeframe.csv')
migration_flows=flow2.iloc[:, 4:147]
migration_flows.to_csv(location_data+'Data\\raw_migration_flows.csv')
mf_df=pd.DataFrame()
mf_df['mig_flow'] = migration_flows.values.tolist()
mf_df['time'] = [pd.to_datetime(list(migration_flows.columns)).values] * len(mf_df)

flow2.to_csv(location_data+'Data\\flow_data_all.csv')
flow2['Country_route_ID'] = flow2.groupby(['Route','Nationality']).grouper.group_info[0]
flow2 = flow2[['Route','Nationality','ISO2','Country_route_ID']]
#TODO: also concat the months in a time-column
#Frames concat:
merged = flow2.join(mf_df)

#set the time bounds:
t_begin = min(merged['time'][0])
t_end = max(merged['time'][0])
b = time_df.reset_index()
b = b[pd.to_datetime(b['date'])>=t_begin]
b = b[pd.to_datetime(b['date'])<=t_end]

#combine columns
keys=b.columns.drop(labels=['date','cat', 'geo', 'gprop'])
b = b.pivot(index='geo', columns='date', values=keys)
for i in keys:
    b[i+'_l'] = b[i].values.tolist()
b = b.drop(columns=keys)
b['GTI'] = b.values.tolist()

#TODO: extend to more than ISO2!!! - Note that we dont have cat and gprop in flow2: we thus need to write some code that increased dimensions of GTI and make e.g. keys like "migration_cat0_gprop"
b = b.reset_index().rename(columns = {'geo':'ISO2'})
b.columns = b.columns.get_level_values(0)
lstm_df = merged.merge(b, on='ISO2')


 ## Add various Fixed Effects

In [9]:
one_hot_countries = pd.get_dummies(lstm_df['ISO2'], prefix='country')
h=pd.DataFrame()
h['Country FE'] = one_hot_countries.values.tolist() 
time_length = np.array(lstm_df['GTI'][0]).shape[1]
def makeTRows(row,T=time_length):
    row['Country FE']=np.array([row['Country FE']]*time_length).T.tolist()
h.apply(makeTRows,axis=1)
lstm_df['Country FE']= h['Country FE']

#Add Route FEs
one_hot_routes = pd.get_dummies(lstm_df['Route'], prefix='route')
h=pd.DataFrame()
h['Route FE'] = one_hot_countries.values.tolist() 
time_length = np.array(lstm_df['GTI'][0]).shape[1]
def makeTRows(row,T=time_length):
    row['Route FE']=np.array([row['Route FE']]*time_length).T.tolist()
h.apply(makeTRows,axis=1)
lstm_df['Route FE']= h['Route FE']

#Add Country-Route FEs
one_hot_routes = pd.get_dummies(lstm_df['Country_route_ID'], prefix='ctrroute')
g=pd.DataFrame()
g['Country-Route FE'] = one_hot_countries.values.tolist() 
time_length = np.array(lstm_df['GTI'][0]).shape[1]
def makeTRows(row,T=time_length):
    row['Country-Route FE']=np.array([row['Country-Route FE']]*time_length).T.tolist()
g.apply(makeTRows,axis=1)
lstm_df['Country-Route FE']= g['Country-Route FE']

#Add time FEs (a TxT unit matrix for all variables):
time_length = len(lstm_df['time'][0])
data_length = len(lstm_df['time'])
lstm_df['Time FE'] = [np.identity(time_length).tolist()] * data_length

#Add lagged dependent variable to later define a superior AR-like model:
def lag_creator_12(row):
    a=pd.DataFrame(row['mig_flow']).shift(12).values.tolist()
    return reduce(lambda x,y: x+y,a)
def lag_creator_1(row):
    a=pd.DataFrame(row['mig_flow']).shift(1).values.tolist()
    return reduce(lambda x,y: x+y,a)
lstm_df['mig_flow_L12']=lstm_df.apply(lag_creator_12, axis=1)
lstm_df['mig_flow_L1']=lstm_df.apply(lag_creator_1, axis=1)

In [10]:
#Save in a pickle and (if wanted) a csv for observation
lstm_df.to_pickle(location_data+'Data\\lstm_df.pkl')
#lstm_df.to_csv(location_data+'Data\\lstm_df.csv')

In [2]:
#Create lagged GTI
def lag_creator_gti_1(row):
    a=pd.DataFrame(row['mig_flow']).shift(1).values.tolist()
    return reduce(lambda x,y: x+y,a)
